In [1]:
import pickle, warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import torch

In [2]:
with open('../../data/processed/gdelt_intermediate_cleaned.pkl', 'rb') as f:
    df = pickle.load(f)

In [3]:
df = df[['GKGRECORDID','article_title']]

In [4]:
# Load model: https://huggingface.co/Qwen/Qwen3-Embedding-0.6B
model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B", model_kwargs={"torch_dtype": "float16"})

In [5]:
prompt = "Instruct: Given a news headline, extract the sentiment that is most likely to affect the airline stock market\nQuery:"

In [6]:
# Encode the article titles with the model
query_embeddings = model.encode(
    df['article_title'], 
    prompt=prompt,
    show_progress_bar=True,
    batch_size=64
)

query_embeddings = np.array(query_embeddings)

Batches: 100%|██████████| 17517/17517 [57:23<00:00,  5.09it/s] 


In [7]:
for i in range(query_embeddings.shape[1]):
    df[f'llm_dimension_{i:04d}'] = query_embeddings[:, i]

In [8]:
# Save df_sentiment to a pickle file
df.drop(columns=['article_title'], inplace=True)

with open('../../data/processed/gdelt_llm_sentiment.pkl', 'wb') as f:
    pickle.dump(df, f)